In [1]:
from pathlib import Path
from typing import List, Tuple, Dict
import json, random, itertools, statistics as st

import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import KFold, train_test_split
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import BallTree

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from seqeval.metrics import f1_score, classification_report
from tqdm.auto import tqdm

/home/hartb/Estudos/mestrado/ner-splits/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configuração e Verificação Inicial

In [2]:
SEED_GLOBAL = 42
random.seed(SEED_GLOBAL)
np.random.seed(SEED_GLOBAL)
torch.manual_seed(SEED_GLOBAL)

CSV_PATH = Path("data/cachacaNER.csv")  # ajuste se necessário
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"

df = pd.read_csv(CSV_PATH)

In [3]:
for cand in ["sentence_id", "sentence", "sent_id"]:
    if cand in df.columns:
        SENT_COL = cand
        break
else:
    raise KeyError(f"Coluna de sentença não encontrada em {list(df.columns)}")


def sent_to_record(sent_id, g):
    return {
        "sentence_id": int(sent_id),
        "tokens": g["token"].tolist(),
        "ner_tags": g["tag"].tolist(),
    }


In [4]:
records = [sent_to_record(i, g) for i, g in df.groupby(SENT_COL, sort=False)]
cachaca_full = Dataset.from_list(records)

In [5]:
cachaca_full

Dataset({
    features: ['sentence_id', 'tokens', 'ner_tags'],
    num_rows: 13628
})

In [6]:
# lista de rótulos (ordem alfabética garante consistência entre runs)
label_list = sorted({lab for sent in cachaca_full["ner_tags"] for lab in sent})
label2id = {l: i for i, l in enumerate(label_list)}
id2label = {i: l for l, i in label2id.items()}
NUM_LABELS = len(label_list)

In [7]:
id2label

{0: 'B-CARACTERISTICA_SENSORIAL_AROMA',
 1: 'B-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA',
 2: 'B-CARACTERISTICA_SENSORIAL_COR',
 3: 'B-CARACTERISTICA_SENSORIAL_SABOR',
 4: 'B-CLASSIFICACAO_BEBIDA',
 5: 'B-EQUIPAMENTO_DESTILACAO',
 6: 'B-GRADUACAO_ALCOOLICA',
 7: 'B-NOME_BEBIDA',
 8: 'B-NOME_LOCAL',
 9: 'B-NOME_ORGANIZACAO',
 10: 'B-NOME_PESSOA',
 11: 'B-PRECO',
 12: 'B-RECIPIENTE_ARMAZENAMENTO',
 13: 'B-TEMPO',
 14: 'B-TEMPO_ARMAZENAMENTO',
 15: 'B-TIPO_MADEIRA',
 16: 'B-VOLUME',
 17: 'I-CARACTERISTICA_SENSORIAL_AROMA',
 18: 'I-CARACTERISTICA_SENSORIAL_CONSISTÊNCIA',
 19: 'I-CARACTERISTICA_SENSORIAL_COR',
 20: 'I-CARACTERISTICA_SENSORIAL_SABOR',
 21: 'I-CLASSIFICACAO_BEBIDA',
 22: 'I-EQUIPAMENTO_DESTILACAO',
 23: 'I-GRADUACAO_ALCOOLICA',
 24: 'I-NOME_BEBIDA',
 25: 'I-NOME_LOCAL',
 26: 'I-NOME_ORGANIZACAO',
 27: 'I-NOME_PESSOA',
 28: 'I-PRECO',
 29: 'I-RECIPIENTE_ARMAZENAMENTO',
 30: 'I-TEMPO',
 31: 'I-TEMPO_ARMAZENAMENTO',
 32: 'I-TIPO_MADEIRA',
 33: 'I-VOLUME',
 34: 'O'}

In [8]:
NUM_LABELS

35

# Splits

In [9]:
def split_standard(ds: Dataset) -> DatasetDict:
    """Usa coluna trainingTest do CSV (80/20 original)."""
    if "trainingTest" not in df.columns:
        raise ValueError("CSV não contém a coluna 'trainingTest'")
    train_ids = df.loc[df["trainingTest"] == "training", SENT_COL].unique()
    test_ids = df.loc[df["trainingTest"] == "test", SENT_COL].unique()
    return DatasetDict(
        train=ds.filter(lambda ex: ex["sentence_id"] in train_ids),
        dev=ds.filter(lambda ex: ex["sentence_id"] in test_ids),
    )

In [10]:
def random_splits(
    ds: Dataset, test_size=0.2, seeds: List[int] = range(30)
) -> List[DatasetDict]:
    triples = []
    for s in seeds:
        train, dev = ds.train_test_split(test_size=test_size, seed=s).values()
        triples.append(DatasetDict(train=train, dev=dev))
    return triples

In [11]:
# def split_heur_length(ds: Dataset, top_pct: float = 0.20) -> DatasetDict:
#     lengths = np.array([len(t) for t in ds["tokens"]])
#     thr = np.percentile(lengths, 100 * (1 - top_pct))
#     mask = lengths >= thr
#     return DatasetDict(train=ds.filter(~mask), dev=ds.filter(mask))


def split_heur_length(ds: Dataset, top_pct: float = 0.20) -> DatasetDict:
    """20 % das sentenças mais longas viram conjunto de validação (dev)."""
    lengths = np.array([len(t) for t in ds["tokens"]])
    thr = np.percentile(lengths, 100 * (1 - top_pct))  
    mask = lengths >= thr  

    dev_idx = np.where(mask)[0].tolist()  # índices → list[int]
    train_idx = np.where(~mask)[0].tolist()

    return DatasetDict(
        train=ds.select(train_idx),
        dev=ds.select(dev_idx),
    )

    # Tamanho da sentenças


# def split_heur_rare(ds: Dataset, freq_thr: int = 5) -> DatasetDict:
#     freq = Counter(w.lower() for sent in ds["tokens"] for w in sent)
#     rare = {w for w, c in freq.items() if c <= freq_thr}

#     def has_rare(example):
#         return any(w.lower() in rare for w in example["tokens"])

#     return DatasetDict(
#         train=ds.filter(lambda ex: not has_rare(ex)), dev=ds.filter(has_rare)
#     )


def split_heur_rare(ds: Dataset, freq_thr: int = 5) -> DatasetDict:
    freq = Counter(w.lower() for sent in ds["tokens"] for w in sent)
    rare = {w for w, c in freq.items() if c <= freq_thr}

    keep_dev = []
    for sent in ds["tokens"]:
        print(sent)
        keep_dev.append(any(w.lower() in rare for w in sent))

    dev_idx = [i for i, x in enumerate(keep_dev) if x]
    train_idx = [i for i, x in enumerate(keep_dev) if not x]

    return DatasetDict(
        train=ds.select(train_idx),
        dev=ds.select(dev_idx),
    )

    # Raridade dos tokens

In [12]:
def split_adversarial(ds: Dataset, pct_test: float = 0.20) -> DatasetDict:
    k = int(len(ds) * pct_test)
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    embeds = model.encode([" ".join(t) for t in ds["tokens"]], show_progress_bar=False)
    tree = BallTree(embeds, leaf_size=40)

    idx_train, idx_test = set(range(len(ds))), []
    # semente = ponto mais central
    seed_idx = np.argmax(np.linalg.norm(embeds - embeds.mean(0), axis=1))
    idx_train.remove(seed_idx)
    idx_test.append(seed_idx)
    print(k)
    while len(idx_test) < k:
        print(len(idx_test))
        dists, _ = tree.query(embeds[list(idx_train)], k=1, return_distance=True)
        nxt = list(idx_train)[int(np.argmax(dists))]
        idx_train.remove(nxt)
        idx_test.append(nxt)

    return DatasetDict(
        train=ds.select(sorted(idx_train)),
        dev=ds.select(sorted(idx_test)),
    )

    # Maximizando Wassertein Distance


def split_adversarial_fast(ds: Dataset, pct_test: float = 0.20) -> DatasetDict:
    """
    Farthest-Point Sampling aproximando Wasserstein – versão vetorizada.
    Seleciona pct_test (~20 %) das sentenças como conjunto 'dev'.
    """
    k = int(len(ds) * pct_test)
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    embeds = model.encode(
        [" ".join(t) for t in ds["tokens"]],
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True,  # acelera distância euclidiana ≈ cos
    )

    n = embeds.shape[0]
    idx_all = np.arange(n)

    # 1) ponto mais "central" (norma mais distante da média)
    seed_idx = np.argmax(np.linalg.norm(embeds - embeds.mean(0), axis=1))
    selected = [seed_idx]

    # 2) vetor de distâncias mínimas a qualquer ponto já escolhido
    min_dists = np.linalg.norm(embeds - embeds[seed_idx], axis=1)

    while len(selected) < k:
        next_idx = np.argmax(min_dists)
        selected.append(next_idx)

        # atualiza min_dists com a distância ao novo ponto — tudo de uma vez
        d_new = np.linalg.norm(embeds - embeds[next_idx], axis=1)
        min_dists = np.minimum(min_dists, d_new)

    train_idx = np.setdiff1d(idx_all, selected, assume_unique=True)

    return DatasetDict(
        train=ds.select(train_idx.tolist()),
        dev=ds.select(selected),
    )

In [13]:
standard_split = split_standard(cachaca_full)
print('std')
random_splt = random_splits(cachaca_full)
print('random')
heur_len = split_heur_length(cachaca_full)
print("heur_len")
heur_rare = split_heur_rare(cachaca_full)
print("heur_rare")
advers = split_adversarial_fast(cachaca_full)
print("advs")

Filter: 100%|██████████| 13628/13628 [00:00<00:00, 29219.89 examples/s]


std
random
heur_len
['NOME', 'DA', 'CACHAÇA', ':', 'Porto', 'Estrela', 'Ouro', '1', 'Litro']
['PREÇO', ':', 'R$', '150', ',', '0']
['DESCRIÇÃO', 'DA', 'CACHAÇA', ':', 'A', 'Cachaça', 'Porto', 'Estrela', 'é', 'produzida', 'na', 'cidade', 'de', 'Pedras', 'de', 'Maria', 'da', 'Cruz', ',', 'Minas', 'Gerais', ',', 'onde', 'é', 'engarrafada', 'em', 'vidro', 'com', 'tampa', 'de', 'rosca', '.']
['Cachaça', 'de', 'grande', 'suavidade', '.']
['Envelhecida', 'em', 'tonéis', 'de', 'madeira', '.']
['Volume', ':', '1000ml']
['Graduação', 'Alcoólica', ':', '42%']
['RECIPIENTE', 'DE', 'ARMAZENAMENTO', ':', 'Tipo', 'Barril', ':', 'Amburana', 'e', 'Carvalho']
['Tipo', ':', 'Envelhecida']
['Produtor', ':', 'Fazenda', 'do', 'Cantagalo']
['Local', 'de', 'Produção', ':', 'Pedras', 'de', 'Maria', 'da', 'Cruz', '/', 'Minas', 'Gerais']
['NOME', 'DA', 'CACHAÇA', ':', 'Gouveia', 'Brasil', 'MEI', '-', 'Cachaça', 'Limao', 'e', 'Mel']
['PREÇO', ':', 'R$', '79', ',', '90']
['DESCRIÇÃO', 'DA', 'CACHAÇA', ':', 'Licor'

Batches: 100%|██████████| 426/426 [00:10<00:00, 39.78it/s] 


advs


# Tokenização e Métricas

In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


# def tokenize(batch):
#     enc = tokenizer(
#         batch["tokens"], is_split_into_words=True, truncation=True, padding=False
#     )
#     new_labels = []
#     for i, word_ids in enumerate(enc.word_ids(batch_index=None)):
#         sent_labels = batch["ner_tags"][i]
#         aligned = []
#         last = None
#         for wid in word_ids:
#             if wid is None:
#                 aligned.append(-100)
#             elif wid != last:
#                 aligned.append(label2id[sent_labels[wid]])
#                 last = wid
#             else:
#                 aligned.append(-100)
#         new_labels.append(aligned)
#     enc["labels"] = new_labels
#     return enc

In [15]:
data_collator = DataCollatorForTokenClassification(
    tokenizer,
    padding=True,  # deixa o collator completar se faltar
)

In [16]:
def tokenize(batch):
    enc = tokenizer(
        batch["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding=True,  # garante mesmo comprimento já aqui
        return_attention_mask=True
    )

    all_labels = []
    for i in range(len(batch["tokens"])):  # cada frase
        word_ids = enc.word_ids(batch_index=i)  # INDEX
        sent_tags = batch["ner_tags"][i]

        aligned = []
        last_wid = None
        for wid in word_ids:
            if wid is None: 
                aligned.append(-100)
            elif wid != last_wid:
                aligned.append(label2id[sent_tags[wid]])
                last_wid = wid
            else:  
                aligned.append(-100)

        all_labels.append(aligned)

    enc["labels"] = all_labels
    return enc

In [17]:
def compute_metrics(p):
    logits, labels = p
    preds = np.argmax(logits, axis=-1)
    true_preds, true_labels = [], []
    for p_i, l_i in zip(preds, labels):
        mask = l_i != -100
        true_preds.append([id2label[idx] for idx in p_i[mask]])
        true_labels.append([id2label[idx] for idx in l_i[mask]])
    return {"f1": f1_score(true_labels, true_preds)}


# def run_experiment(dsdict: DatasetDict, seed: int) -> float:
#     encoded = dsdict.map(
#         tokenize, batched=True, remove_columns=dsdict["train"].column_names
#     )
#     model = AutoModelForTokenClassification.from_pretrained(
#         MODEL_NAME, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id
#     )
#     args = TrainingArguments(
#         output_dir=f"tmp/seed{seed}",
#         eval_strategy="no",
#         logging_strategy="steps",
#         logging_steps=50,  # → ≥ 1
#         learning_rate=2e-5,
#         per_device_train_batch_size=16,
#         num_train_epochs=3,
#         seed=seed,
#         save_strategy="no",
#         report_to="none",
#     )
#     trainer = Trainer(
#         model=model,
#         args=args,
#         data_collator=data_collator,
#         train_dataset=encoded["train"],
#         compute_metrics=compute_metrics,
#     )
#     trainer.train()
#     return trainer.evaluate(encoded["dev"])["eval_f1"]


def run_experiment(dsdict: DatasetDict, seed: int) -> float:
    encoded = dsdict.map(
        tokenize, batched=True, remove_columns=dsdict["train"].column_names
    )

    model = AutoModelForTokenClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        id2label=id2label,
        label2id=label2id,
        torch_dtype="auto",  # usa fp16 se possível
    )

    args = TrainingArguments(
        output_dir=f"tmp/seed{seed}",
        eval_strategy="no",
        learning_rate=2e-5,
        per_device_train_batch_size=2,  # <<< menor
        gradient_accumulation_steps=8,  # <<< compensa
        num_train_epochs=3,
        fp16=True,  # tente first; se falhar, tire
        seed=seed,
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="no",
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=args,
        data_collator=data_collator,  # já sugerido p/ padding labels
        train_dataset=encoded["train"],
        compute_metrics=compute_metrics,
    )
    trainer.train()
    return trainer.evaluate(encoded["dev"])["eval_f1"]

# Experimentos

In [18]:
# print("• Standard split")
# std_f1 = run_experiment(standard_split, seed=SEED_GLOBAL)

In [ ]:
print("• 30 random splits")
rand_scores = [
    run_experiment(split, seed=i)  # split = DatasetDict
    for i, split in enumerate(random_splt)
]

• 30 random splits


Map:   0%|          | 0/10902 [00:00<?, ? examples/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map: 100%|██████████| 2726/2726 [00:00<00:00, 8263.21 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  2%|▏         | 50/2043 [03:56<2:37:25,  4.74s/it]

{'loss': 1.3977, 'grad_norm': 1.653882384300232, 'learning_rate': 1.951052373959863e-05, 'epoch': 0.07}


  5%|▍         | 100/2043 [07:50<2:28:54,  4.60s/it]

...

  7%|▋         | 150/2043 [12:01<2:34:00,  4.88s/it]

{'loss': 0.3878, 'grad_norm': 2.8727803230285645, 'learning_rate': 1.8541360744003917e-05, 'epoch': 0.22}


 10%|▉         | 200/2043 [16:00<2:27:02,  4.79s/it]

{'loss': 0.3125, 'grad_norm': 1.6727875471115112, 'learning_rate': 1.8051884483602547e-05, 'epoch': 0.29}


 12%|█▏        | 247/2043 [19:52<2:31:16,  5.05s/it]

In [ ]:
rand_scores

In [ ]:
import statistics as st

print(f"Random  mean={st.mean(rand_scores):.3f}  sd={st.stdev(rand_scores):.3f}")

In [ ]:
# print("• Heuristic length")
# f1_len = run_experiment(heur_len, seed=SEED_GLOBAL)

• Heuristic length


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 2813/2813 [00:00<00:00, 6174.18 examples/s]
Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                        
  0%|          | 0/2028 [02:53<?, ?it/s]           

{'loss': 1.4888, 'grad_norm': 2.4475419521331787, 'learning_rate': 1.95069033530572e-05, 'epoch': 0.07}


                                        
  0%|          | 0/2028 [04:33<?, ?it/s]            

{'loss': 0.5489, 'grad_norm': 2.2895572185516357, 'learning_rate': 1.90138067061144e-05, 'epoch': 0.15}


...

                                          
  0%|          | 0/2028 [1:08:29<?, ?it/s]           

{'loss': 0.0472, 'grad_norm': 1.6855499744415283, 'learning_rate': 2.7613412228796843e-07, 'epoch': 2.96}


                                          
100%|██████████| 2028/2028 [1:08:11<00:00,  2.02s/it]


{'train_runtime': 4091.9232, 'train_samples_per_second': 7.929, 'train_steps_per_second': 0.496, 'train_loss': 0.1421811213860145, 'epoch': 3.0}


100%|██████████| 352/352 [04:18<00:00,  1.36it/s]


In [ ]:


# print("• Heuristic rare")
# f1_rare = run_experiment(heur_rare, seed=SEED_GLOBAL)

# print("• Adversarial")
# f1_adv = run_experiment(advers, seed=SEED_GLOBAL)

In [ ]:
print("\n=====  RESULTADOS  =====")
# print(f"Standard F1 : {std_f1:.3f}")
print(f"Random   F1 : {rand_f1:.3f}")
# print(f"Heur-len F1 : {f1_len:.3f}")
# print(f"Heur-rare F1: {f1_rare:.3f}")
# print(f"Advers.  F1 : {f1_adv:.3f}")


=====  RESULTADOS  =====
Heur-len F1 : 0.830


In [ ]:
# from transformers import TrainingArguments
# import inspect, os, pathlib

# print("TrainingArguments vindo de:", inspect.getfile(TrainingArguments))
# print(
#     "Tem avaliação?:",
#     "evaluation_strategy" in inspect.signature(TrainingArguments).parameters,
# )